# PRÉ-AMBULO - IMPORT LIBS

In [2]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
import torchvision
import torch
import cv2
import os
import glob
import numpy as np

from modules.helper.helper import create_video_from_images, get_framerate

# V1 - Background Subtractor

In [ ]:
# Caminho para o vídeo
video_path  = "input\\Teste.mp4"
save        = "output\\sub_background"
os.makedirs(save, exist_ok=True)

# Captura de vídeo
cap = cv2.VideoCapture(video_path)

prev_gray = None
idx = 0

img_before  = None
background_stack = []  # Para armazenar frames sem movimento

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if type(img_before) != type(None):
        
        # Grayscale e equalização
        gray_prev = cv2.equalizeHist(cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY))
        gray_curr = cv2.equalizeHist(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        # Redução de ruído
        gray_prev = cv2.GaussianBlur(gray_prev, (5, 5), 0)
        gray_curr = cv2.GaussianBlur(gray_curr, (5, 5), 0)

        # Subtração e limiarização
        diff = cv2.absdiff(gray_prev, gray_curr)
        _, motion_mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Remover pequenos ruídos
        motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

        # Inverter para pegar áreas "estáticas"
        static_mask = cv2.bitwise_not(motion_mask)

        # Aplicar máscara estática ao frame atual
        static_pixels = cv2.bitwise_and(frame, frame, mask=static_mask)
        background_stack.append(static_pixels)

        # Salvar máscara de movimento
        cv2.imwrite(os.path.join(save, f"results{idx}.jpg"), motion_mask)

    img_before = frame.copy()
    idx+=1

# # Gerar background final pela mediana dos frames estáticos
# if background_stack:
#     background_array = np.stack(background_stack, axis=3)
#     background_final = np.median(background_array, axis=3).astype(np.uint8)
#     cv2.imwrite(os.path.join("background_final.jpg"), background_final)

# Exemplo de uso
image_folder = save
output_video_path = "sub_background.mp4"
frame_rate = 13  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")

Vídeo criado com sucesso: sub_background.mp4


0

# V2 - calcOpticalFlowFarneback

In [8]:
# Caminho para o vídeo
video_path  = "input\\Teste.mp4"
save        = "output\\optical_flow"
os.makedirs(save, exist_ok=True)

# Captura de vídeo
cap = cv2.VideoCapture(video_path)

prev_gray = None
idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_gray is not None:
        
        # Calcula o fluxo óptico denso
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray,      # imagem anterior (grayscale)
            gray,           # imagem atual (grayscale)
            None,           # matriz de fluxo (pode ser None)
            pyr_scale=0.25, # escala da pirâmide de imagem
            levels=3,       # número de níveis da pirâmide
            winsize=10,     # tamanho da janela
            iterations=5,   # número de iterações por nível
            poly_n=7,       # tamanho da vizinhança para polinômio
            poly_sigma=1.5, # sigma (desvio padrão) da Gaussiana usada
            flags=0         # opções adicionais (geralmente 0)
        )

        # Converte o fluxo em magnitude e ângulo
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Cria máscara binária para movimento baseado na magnitude
        movement_mask = cv2.threshold(mag, 1.5, 255, cv2.THRESH_BINARY)[1]
        movement_mask = movement_mask.astype(np.uint8)

        # Salva a máscara de movimento
        cv2.imwrite(os.path.join(save, f"results{idx}.jpg"), movement_mask)

        # # (Opcional) Mostra o vetor de movimento com cor HSV
        # hsv = np.zeros_like(frame)
        # hsv[..., 1] = 255
        # hsv[..., 0] = ang * 180 / np.pi / 2
        # hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        # flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        # cv2.imwrite(os.path.join(save, f"flow_visual_{idx:04}.jpg"), flow_rgb)

    prev_gray = gray
    idx+=1

# Exemplo de uso
image_folder = save
output_video_path = "motion_flow.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")

Vídeo criado com sucesso: motion_flow.mp4


0

# V3 - createBackgroundSubtractorKNN

In [21]:
# Caminhos
video_path = "input\\Teste.mp4"
save = "output\\knn_video"
os.makedirs(save, exist_ok=True)

# Inicializar captura e detector KNN
cap = cv2.VideoCapture(video_path)
fgbg = cv2.createBackgroundSubtractorKNN(history=30, dist2Threshold=400, detectShadows=False)

# Kernels para refinamento de máscaras
kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
kernel_close = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))

area = 100  # Área mínima para contornos (ajustar conforme necessário)

idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Ajuste de constraste
    frame_blur = cv2.GaussianBlur(frame, (3, 3), 0)
    frame_detail = cv2.detailEnhance(frame_blur, sigma_s=30, sigma_r=0.1)

    # Grayscale e equalização
    frame_equalized = cv2.equalizeHist(cv2.cvtColor(frame_detail, cv2.COLOR_BGR2GRAY))
    
    # Redução de ruído
    frame_blur = cv2.GaussianBlur(frame_equalized, (3, 3), 0)

    # Limpeza de ruído (leve e rápida)
    frame_open = cv2.morphologyEx(frame_blur, cv2.MORPH_OPEN, kernel_open, iterations=1)
    frame_close = cv2.morphologyEx(frame_open, cv2.MORPH_CLOSE, kernel_close, iterations=2)
    
    # Aplicar subtração de fundo KNN
    fgmask = fgbg.apply(frame_close)

    # Detectar contornos dos objetos claramente definidos
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Criar máscara vazia para contornos precisos
    precise_mask = np.zeros_like(fgmask)

    # Preencher contornos na máscara
    for cnt in contours:
        # Ignorar pequenos ruídos
        if cv2.contourArea(cnt) >= area:  
            cv2.drawContours(precise_mask, [cnt], -1, 255, -1)  # Preenche o objeto detectado
        else:
            cv2.drawContours(precise_mask, [cnt], -1, 0, -1)  # Preenche o objeto detectado

    # Extrair objetos em movimento (sem borrões)
    moving_objects = cv2.bitwise_and(frame, frame, mask=precise_mask)

    # Opcional: Desenhar caixa delimitadora ao redor dos objetos (facilita identificação)
    for cnt in contours:
        if cv2.contourArea(cnt) >= area:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Salvar resultado com objetos definidos
    cv2.imwrite(os.path.join(save, f"results_{idx:04}.jpg"), frame)

    idx += 1
    
cap.release()

# Criar vídeo final
image_folder = save
output_video_path = "motion_knn.mp4"
frame_rate = get_framerate(video_path)
create_video_from_images(image_folder, output_video_path, frame_rate)

# Abrir vídeo automaticamente após criação
os.system(f"start {output_video_path}")

Vídeo criado com sucesso: motion_knn.mp4


0

# V4 - Motrackers

In [ ]:
!pip install opencv-contrib-python==4.5.3.56

In [ ]:
cap = cv2.VideoCapture("input\\Teste.mp4")

# Trackers atualizados para a versão atual do OpenCV
OPENCV_OBJECT_TRACKERS = {
    "csrt":         cv2.legacy.TrackerCSRT_create,
    # "kcf":          cv2.legacy.TrackerKCF_create,
    "boosting":     cv2.legacy.TrackerBoosting_create,
    # "mil":          cv2.legacy.TrackerMIL_create,
    "tld":          cv2.legacy.TrackerTLD_create,
    "medianflow":   cv2.legacy.TrackerMedianFlow_create,
    "mosse":        cv2.legacy.TrackerMOSSE_create
}

# Agora gerenciamos múltiplos trackers manualmente
trackers = []

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.resize(frame, (1090, 600))

    boxes = []
    new_trackers = []

    for tracker in trackers:
        success, box = tracker.update(frame)
        if success:
            boxes.append(box)
            new_trackers.append(tracker)

    trackers = new_trackers

    # Desenha caixas atualizadas
    for box in boxes:
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, 'TRACKING', (x + 10, y - 3), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 0), 2)

    cv2.imshow('Frame', frame)
    k = cv2.waitKey(30)

    if k == ord("s"):
        roi = cv2.selectROI("Frame", frame, fromCenter=False, showCrosshair=True)
        tracker = OPENCV_OBJECT_TRACKERS['tld']()
        tracker.init(frame, roi)
        trackers.append(tracker)

    if k == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.5/1.5 MB 6.0 MB/s eta 0:00:00

requirements: AutoUpdate success  5.7s, installed 1 package: ['lap>=0.5.12']
requirements:  Restart runtime or rerun command for updates to take effect


0: 288x512 6 cars, 209.4ms
Speed: 11.7ms preprocess, 209.4ms inference, 15.1ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 6 cars, 103.4ms
Speed: 3.5ms preprocess, 103.4ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 6 cars, 93.6ms
Speed: 2.9ms preprocess, 93.6ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 6 cars, 174.3ms
Speed: 1.9ms preprocess, 174.3ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 6 cars, 87.0ms
Speed: 2.0ms preprocess, 87.0ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 6 cars, 88.9ms
Speed: 1.5ms preprocess, 

# SPLIT VIDEO TO IMAGE

In [10]:
# Caminhos
video_path = "input\\Recv02_20250325135451@5802d41e.avi"
save = "output\\splited"
os.makedirs(save, exist_ok=True)

frame_rate = get_framerate(video_path) * 2 # Obter o frame rate do vídeo

# Inicializar captura
cap = cv2.VideoCapture(video_path)
idx = 0
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Verifica se o frame atual é o primeiro frame do segundo
    if frame_count % frame_rate == 0:
        # Salvar o frame
        cv2.imwrite(os.path.join(save, f"results_{idx:04}.jpg"), frame)
        idx += 1

    frame_count += 1

cap.release()

In [4]:
get_framerate(video_path)

25